In [6]:
import difflib


def longest_common_contiguous_subsequence_difflib(list1, list2):
    """
    Finds the longest common contiguous subsequence between two lists using difflib.

    Args:
        list1 (list): The first list.
        list2 (list): The second list.

    Returns:
        tuple: A tuple containing:
            - overlap_seq (list): The longest common contiguous subsequence.
            - start1 (int): The starting index of the subsequence in list1.
            - start2 (int): The starting index of the subsequence in list2.
            - length (int): The length of the subsequence.
            - direction (str): 'suffix_prefix' or 'prefix_suffix' based on overlap.
    """
    matcher = difflib.SequenceMatcher(None, list1, list2)
    matching_blocks = matcher.get_matching_blocks()
    # Exclude the dummy block at the end
    matching_blocks = [block for block in matching_blocks if block.size > 0]

    if not matching_blocks:
        return [], -1, -1, 0, None

    # Find the block with the maximum size
    max_block = max(matching_blocks, key=lambda block: block.size)
    overlap_seq = list1[max_block.a : max_block.a + max_block.size]
    start1 = max_block.a
    start2 = max_block.b
    length = max_block.size

    # Determine the direction of overlap
    direction = None
    if (start1 + length == len(list1)) and (start2 == 0):
        direction = "suffix_prefix"
    elif (start1 == 0) and (start2 + length == len(list2)):
        direction = "prefix_suffix"
    else:
        # Default to 'suffix_prefix' if not clearly one of the above
        if start1 + length == len(list1):
            direction = "suffix_prefix"
        elif start1 == 0:
            direction = "prefix_suffix"
        else:
            direction = "suffix_prefix"  # Fallback

    return overlap_seq, start1, start2, length, direction


def rotate_lists_to_align_overlap(
    list1, list2, overlap_seq, start1, start2, length, direction
):
    """
    Rotates list1 and list2 based on the overlap direction.

    Args:
        list1 (list): First list.
        list2 (list): Second list.
        overlap_seq (list): The overlapping sequence.
        start1 (int): Starting index in list1.
        start2 (int): Starting index in list2.
        length (int): Length of the overlap.
        direction (str): 'suffix_prefix' or 'prefix_suffix'

    Returns:
        tuple: (list1_rotated, list2_rotated)
    """
    if length > 0:
        if direction == "suffix_prefix":
            # Rotate list1: remove the overlapping part and append the overlap
            list1_rotated = list1[:start1] + overlap_seq
            # Rotate list2: prepend the overlap and remove it from its original position
            list2_rotated = overlap_seq + list2[start2 + length :]
        elif direction == "prefix_suffix":
            # Rotate list1: remove the overlapping part and append the overlap
            list1_rotated = list1[start1 + length :] + overlap_seq
            # Rotate list2: prepend the overlap and retain the preceding part
            list2_rotated = overlap_seq + list2[:start2]
    else:
        list1_rotated = list1
        list2_rotated = list2

    return list1_rotated, list2_rotated


def merge_rotated_lists(list1_rotated, list2_rotated, overlap_len, direction):
    """
    Merges two rotated lists, ensuring that overlapping sequences are not duplicated.

    Args:
        list1_rotated (list): The rotated first list.
        list2_rotated (list): The rotated second list.
        overlap_len (int): The length of the overlap sequence.
        direction (str): 'suffix_prefix' or 'prefix_suffix'

    Returns:
        list: The merged list with no duplicated overlaps.
    """
    if overlap_len > 0:
        if direction == "suffix_prefix":
            # Remove overlapping elements from list2_rotated to avoid duplication
            return list1_rotated + list2_rotated[overlap_len:]
        elif direction == "prefix_suffix":
            # Remove overlapping elements from list2_rotated to avoid duplication
            return list1_rotated + list2_rotated[overlap_len:]
    else:
        return list1_rotated + list2_rotated


def construct_nested_and_merged(input_lists):
    """
    Given two lists, finds the longest overlapping subsequence, rotates the lists to align the overlap, and merges them.

    Args:
        input_lists (list of list): A list containing two lists.

    Returns:
        tuple: A tuple containing:
            - nested_rotated (list of list): The two rotated lists.
            - merged (list): The merged list without duplicate overlapping elements.
    """
    list1, list2 = input_lists

    # Step 1: Find the longest common contiguous subsequence using difflib
    overlap_seq, start1, start2, overlap_len, direction = (
        longest_common_contiguous_subsequence_difflib(list1, list2)
    )

    # Step 2: Rotate both lists to align the overlap sequence
    list1_rotated, list2_rotated = rotate_lists_to_align_overlap(
        list1, list2, overlap_seq, start1, start2, overlap_len, direction
    )

    # Step 3: Merge the rotated lists, avoiding duplicate overlaps
    merged = merge_rotated_lists(list1_rotated, list2_rotated, overlap_len, direction)

    return [list1_rotated, list2_rotated], merged


def run_tests(test_cases):
    """
    Runs a set of test cases for the nested and merged lists.

    Args:
        test_cases (list of dict): A list of test case dictionaries.
    """
    all_passed = True
    for idx, test in enumerate(test_cases, 1):
        input_lists = test["input"]
        expected_nested = test["expected_nested"]
        expected_merged = test["expected_merged"]

        output_nested, output_merged = construct_nested_and_merged(input_lists)

        nested_pass = output_nested == expected_nested
        merged_pass = output_merged == expected_merged

        if nested_pass and merged_pass:
            print(f"Test Case {idx}: PASSED")
        else:
            all_passed = False
            print(f"Test Case {idx}: FAILED")
            print(f"  Input: {input_lists}")
            print(f"  Expected Nested: {expected_nested}")
            print(f"  Output Nested:   {output_nested}")
            print(f"  Expected Merged: {expected_merged}")
            print(f"  Output Merged:   {output_merged}")

    if all_passed:
        print("\nAll test cases passed successfully!")
    else:
        print("\nSome test cases failed. Please review the output above.")


if __name__ == "__main__":
    # Test cases provided by the user
    test_cases = [
        {
            "input": [["B", "C", "A"], ["A", "E", "F", "D"]],
            "expected_nested": [["B", "C", "A"], ["A", "E", "F", "D"]],
            "expected_merged": ["B", "C", "A", "E", "F", "D"],
        },
        {
            "input": [["A", "B", "C"], ["E", "F", "D", "A"]],
            "expected_nested": [["B", "C", "A"], ["A", "E", "F", "D"]],
            "expected_merged": ["B", "C", "A", "E", "F", "D"],
        },
        {
            "input": [["A", "B", "C"], ["A", "E", "F", "D"]],
            "expected_nested": [["B", "C", "A"], ["A", "E", "F", "D"]],
            "expected_merged": ["B", "C", "A", "E", "F", "D"],
        },
        # Additional test cases to ensure robustness
        {
            "input": [["X", "Y", "Z"], ["Z", "A", "B"]],
            "expected_nested": [["X", "Y", "Z"], ["Z", "A", "B"]],
            "expected_merged": ["X", "Y", "Z", "A", "B"],
        },
        # Edge case with partial overlap (suffix-prefix)
        {
            "input": [["A", "B", "C", "D"], ["C", "D", "E", "F"]],
            "expected_nested": [["A", "B", "C", "D"], ["C", "D", "E", "F"]],
            "expected_merged": ["A", "B", "C", "D", "E", "F"],
        },
        # Edge case where list2 starts with multiple overlapping elements (prefix-suffix)
        {
            "input": [["A", "B", "C", "D"], ["B", "C", "D", "E"]],
            "expected_nested": [["A", "B", "C", "D"], ["B", "C", "D", "E"]],
            "expected_merged": ["A", "B", "C", "D", "E"],
        },
        # Newly added test case (prefix-suffix overlap)
        {
            "input": [["B", "C", "B1", "B2"], ["B1", "B2", "E", "F", "D"]],
            "expected_nested": [["B", "C", "B1", "B2"], ["B1", "B2", "E", "F", "D"]],
            "expected_merged": ["B", "C", "B1", "B2", "E", "F", "D"],
        },
        # No overlap
        {
            "input": [["A", "B", "C"], ["D", "E", "F"]],
            "expected_nested": [["A", "B", "C"], ["D", "E", "F"]],
            "expected_merged": ["A", "B", "C", "D", "E", "F"],
        },
        # Complete overlap
        {
            "input": [["A", "B", "C"], ["A", "B", "C"]],
            "expected_nested": [["A", "B", "C"], ["A", "B", "C"]],
            "expected_merged": ["A", "B", "C"],
        },
    ]

    print("Running tests using difflib-based LCCS implementation:")
    run_tests(test_cases)

Running tests using difflib-based LCCS implementation:
Test Case 1: PASSED
Test Case 2: PASSED
Test Case 3: FAILED
  Input: [['A', 'B', 'C'], ['A', 'E', 'F', 'D']]
  Expected Nested: [['B', 'C', 'A'], ['A', 'E', 'F', 'D']]
  Output Nested:   [['B', 'C', 'A'], ['A']]
  Expected Merged: ['B', 'C', 'A', 'E', 'F', 'D']
  Output Merged:   ['B', 'C', 'A']
Test Case 4: PASSED
Test Case 5: PASSED
Test Case 6: PASSED
Test Case 7: PASSED
Test Case 8: PASSED
Test Case 9: PASSED

Some test cases failed. Please review the output above.


In [5]:
from brancharchitect.io import read_newick


# Function to check if element_x is a proper subset of any element in the list
def check_if_not_subset(full_combination, element_x):
    for element_y in full_combination:
        if set(element_x).issubset(element_y) and element_x != element_y:
            return False  # element_x is a subset of another element
    return True  # element_x is unique or not a subset of any other element


tree_list = read_newick("./../data/alltrees_treees_cutted/alltrees.trees_cutted.newick")
# min_perm = generate_permutations_on_randomness(tree_list)
# tree_list = read_newick("./../data/toy_example.newick", min_perm)

tree_one = tree_list[0]
tree_two = tree_list[1]

splits1, splits2 = tree_one.to_splits(), tree_two.to_splits()

set1 = set(splits1)
set2 = set(splits2)

total_a_without_b = set1 - set2  # Symmetric difference
total_b_without_a = set2 - set1  # Symmetric difference
total_unique_splits = set1.intersection(set2)  # Union of both sets

print("A without B")
for split in total_a_without_b:
    print([tree_one._order[i] for i in split])

print("B without A")
for split in total_b_without_a:
    print([tree_one._order[i] for i in split])

print("Full Combinaition")
full_combination = []
for split in total_a_without_b:
    full_combination.append([tree_one._order[i] for i in split])
for split in total_b_without_a:
    full_combination.append([tree_one._order[i] for i in split])

filtered_elements = []
for element_x in full_combination:
    if check_if_not_subset(full_combination, element_x):
        filtered_elements.append(element_x)

print("Filtered elements", filtered_elements)

A without B
B without A
Full Combinaition
Filtered elements []
